Третью моделью стало дерево решений я подумал вдруг есть особенные признаки, которые хорошо разделят данные, как оно работает:
алгоритм перебирает все возможные признаки и все возможные способы разделить данные по этому признаку.
Цель: Найти такой признак и такое правило разделения, которое максимально "очистит" получившиеся группы. То есть, в идеале, после разделения в каждой группе должны остаться классы в основном одного типа.

In [3]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import warnings
from sklearn.metrics import accuracy_score, f1_score
import time
warnings.filterwarnings('ignore')
import pandas as pd
from sklearn.model_selection import train_test_split

train = pd.read_csv('train.csv')
split_idx = int(0.8 * len(train))
test = train.iloc[split_idx:]
train = train.iloc[:split_idx]

X_train = train.drop('Fertilizer Name', axis=1)
y_train = train['Fertilizer Name']

X_test = test.drop('Fertilizer Name', axis=1)
y_test = test['Fertilizer Name']


from sklearn.preprocessing import OneHotEncoder
cat_cols = ['Soil Type', 'Crop Type']
encoder = OneHotEncoder(handle_unknown='ignore')
X_train_encoded = encoder.fit_transform(X_train[cat_cols]).toarray()
feature_names = encoder.get_feature_names_out(cat_cols)
X_test_encoded = encoder.transform(X_test[cat_cols]).toarray()

from sklearn.preprocessing import StandardScaler
num_cols = ['Temparature', 'Humidity', 'Moisture', 'Nitrogen', 'Potassium', 'Phosphorous']
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train[num_cols])
X_test_scaled = scaler.transform(X_test[num_cols])

X_train_processed = np.hstack([X_train_scaled, X_train_encoded])
X_test_processed = np.hstack([X_test_scaled, X_test_encoded])

results = {}

In [4]:
print("\nОбучение Decision Tree...")
start_time = time.time()
dt = DecisionTreeClassifier(max_depth=5, random_state=42)
dt.fit(X_train_processed, y_train)
y_pred = dt.predict(X_test_processed)
results["Decision Tree"] = {
    'Accuracy': accuracy_score(y_test, y_pred),
    'F1 Score': f1_score(y_test, y_pred, average='weighted')
}
print(f"Завершено за {time.time() - start_time:.2f} сек. F1: {results['Decision Tree']['F1 Score']:.4f}")


Обучение Decision Tree...
Завершено за 8.74 сек. F1: 0.1074


Что-то вообще грустно, но попробуем подобрать гипер парамеры при условии что модель обучилась быстро сделаем это перебором

как всегда давайте посмотрим за что отвечает каждый параметр модели
max_depth - макс глубина Насколько длинными могут быть цепочки вопросов

min_samples_split - Сколько примеров нужно в узле, чтобы его можно было разделить

min_samples_leaf  - Сколько примеров должно быть в конечном пункте

criterion - Как измерять качество разделения данных

In [8]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import time

param_grid = {
    'max_depth': [3, 5, 7, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['gini', 'entropy']
}

print("\nПодбор гиперпараметров для Decision Tree...")
start_time = time.time()
dt_search = GridSearchCV(
    DecisionTreeClassifier(random_state=42),
    param_grid,
    cv=3,
    scoring='f1_weighted',
    n_jobs=-1
)
dt_search.fit(X_train_processed, y_train)
best_dt = dt_search.best_estimator_
y_pred = best_dt.predict(X_test_processed)

results["Decision Tree (optimized)"] = {
    'Accuracy': accuracy_score(y_test, y_pred),
    'Precision': precision_score(y_test, y_pred, average='weighted'),
    'Recall': recall_score(y_test, y_pred, average='weighted'),
    'F1 Score': f1_score(y_test, y_pred, average='weighted')
}

print(f"Лучшие параметры: {dt_search.best_params_}")
print(f"Завершено за {time.time() - start_time:.2f} сек.")
print(f"Accuracy: {results['Decision Tree (optimized)']['Accuracy']:.4f}")
print(f"Precision: {results['Decision Tree (optimized)']['Precision']:.4f}")
print(f"Recall: {results['Decision Tree (optimized)']['Recall']:.4f}")
print(f"F1: {results['Decision Tree (optimized)']['F1 Score']:.4f}")


Подбор гиперпараметров для Decision Tree...
Лучшие параметры: {'criterion': 'gini', 'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 10}
Завершено за 935.85 сек.
Accuracy: 0.1541
Precision: 0.1528
Recall: 0.1541
F1: 0.1526
